In [11]:
pip install qiskit


In [12]:
pip install qiskit-aer

In [13]:
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import qiskit_aer


In [14]:
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import qiskit_aer
import numpy as np

def biased_qgb(num_rows, theta=np.pi/4):
    from qiskit.circuit import QuantumRegister, ClassicalRegister
    from qiskit.circuit.library.standard_gates import RXGate, CSwapGate

    # Define registers
    coin = QuantumRegister(1, 'coin')
    position = QuantumRegister(num_rows + 1, 'pos')
    classical = ClassicalRegister(num_rows + 1, 'c')
    qc = QuantumCircuit(coin, position, classical)

    # Initialize ball in the center
    middle_index = num_rows // 2
    qc.x(position[middle_index])

    # Loop through each row (each has one peg interaction)
    for row in range(num_rows):
        # Reset coin qubit
        qc.reset(coin)

        # Biased coin toss
        qc.append(RXGate(theta), [coin[0]])

        # Conditional movement using CSWAP
        if row < num_rows:
            qc.append(CSwapGate(), [coin[0], position[row], position[row + 1]])

        # Peg interaction
        qc.cx(coin[0], position[row])

        # Corrective CNOT and RESET for rows > 0
        for j in range(row):
            qc.cx(coin[0], position[j])
            qc.reset(position[j])

        qc.barrier()

    # Measure position qubits
    qc.measure(position, classical)

    return qc

# Parameters
num_rows = 5
theta = np.pi / 3  # Biased angle

# Build circuit
qc = biased_qgb(num_rows, theta)
print(qc.draw())
plt.show()

# Simulate with qiskit_aer
simulator = qiskit_aer.AerSimulator()
tqc = transpile(qc, simulator)
result = simulator.run(tqc, shots=2048).result()
counts = result.get_counts()

# Convert to position distribution
position_distribution = {}
for bitstring, count in counts.items():
    pos = bitstring[::-1]  # Qiskit uses little-endian bit order
    index = pos.find('1')  # Position of the ball
    if index != -1:
        position_distribution[index] = position_distribution.get(index, 0) + count

# Plot result
plot_histogram(position_distribution, title="Fine-Grained Biased Quantum Galton Board")
plt.show()


            ┌─────────┐         ░      ┌─────────┐                   ░      »
 coin: ─|0>─┤ Rx(π/3) ├─■───■───░──|0>─┤ Rx(π/3) ├─■───■────■────────░──|0>─»
            └─────────┘ │ ┌─┴─┐ ░      └─────────┘ │   │  ┌─┴─┐      ░      »
pos_0: ─────────────────X─┤ X ├─░──────────────────┼───┼──┤ X ├─|0>──░──────»
                        │ └───┘ ░                  │ ┌─┴─┐└───┘      ░      »
pos_1: ─────────────────X───────░──────────────────X─┤ X ├───────────░──────»
       ┌───┐                    ░                  │ └───┘           ░      »
pos_2: ┤ X ├────────────────────░──────────────────X─────────────────░──────»
       └───┘                    ░                                    ░      »
pos_3: ─────────────────────────░────────────────────────────────────░──────»
                                ░                                    ░      »
pos_4: ─────────────────────────░────────────────────────────────────░──────»
                                ░                               